## A intenção do projeto é criar um chatbot baseado em reviews de filmes para que se possa fazer perguntas e manter uma conversa livre

- link do banco de dados https://www.kaggle.com/Cornell-University/movie-dialog-corpus?select=movie_lines.tsv
- referências
>- https://shanebarker.com/blog/deep-learning-chatbot/
> -https://towardsdatascience.com/how-to-create-a-chatbot-with-python-deep-learning-in-less-than-an-hour-56a063bdfc44

In [2]:
import pandas as pd
import re
import gensim
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
#setting parameters for data visualization
np.set_printoptions(threshold=None, precision=2)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('precision', 2)

### Opening movie reviews

In [5]:
messages = pd.read_csv('./chatdata/movie_titles_metadata.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [6]:
messages.columns = ['movie_id', 'movie_name', 'movie_age', 'movie_rating', 'qtde_votes', 'genre']

In [7]:
messages.head(10)

,movie_id,movie_name,movie_age,movie_rating,qtde_votes,genre
0,m0,10 things i hate about you,1999,6.9,62847,['comedy' 'romance']
1,m1,1492: conquest of paradise,1992,6.2,10421,['adventure' 'biography' 'drama' 'history']
2,m2,15 minutes,2001,6.1,25854,['action' 'crime' 'drama' 'thriller']
3,m3,2001: a space odyssey,1968,8.4,163227,['adventure' 'mystery' 'sci-fi']
4,m4,48 hrs.,1982,6.9,22289,['action' 'comedy' 'crime' 'drama' 'thriller']
5,m5,the fifth element,1997,7.5,133756,['action' 'adventure' 'romance' 'sci-fi' 'thri...
6,m6,8mm,1999,6.3,48212,['crime' 'mystery' 'thriller']
7,m7,a nightmare on elm street 4: the dream master,1988,5.2,13590,['fantasy' 'horror' 'thriller']
8,m8,a nightmare on elm street: the dream child,1989,4.7,11092,['fantasy' 'horror' 'thriller']
9,m9,the atomic submarine,1959,4.9,513,['sci-fi' 'thriller']


### Getting the genre (topics)

In [10]:
genre = list(messages['genre'])

In [27]:
genre[0].split(' ')

["['comedy'", "'romance']"]

In [31]:
li = list()
for g in genre:
    g = g.split(' ')
    for h in g:
        #li.append(h)
        print(h)
    
print(li)

['comedy'
'romance']
['adventure'
'biography'
'drama'
'history']
['action'
'crime'
'drama'
'thriller']
['adventure'
'mystery'
'sci-fi']
['action'
'comedy'
'crime'
'drama'
'thriller']
['action'
'adventure'
'romance'
'sci-fi'
'thriller']
['crime'
'mystery'
'thriller']
['fantasy'
'horror'
'thriller']
['fantasy'
'horror'
'thriller']
['sci-fi'
'thriller']
['drama'
'mystery'
'thriller']
['action'
'drama'
'thriller']
['comedy'
'romance'
'sci-fi']
['comedy'
'romance']
['crime'
'drama'
'sci-fi'
'thriller']
['action'
'sci-fi'
'thriller']
['biography'
'drama'
'music']
['horror'
'romance']
['drama']
['action'
'western']
['drama'
'thriller']
['drama'
'thriller']
['action'
'adventure'
'comedy'
'crime']
['action'
'adventure'
'thriller']
['comedy'
'romance']
['action'
'crime'
'drama'
'mystery'
'thriller']
['crime'
'drama']
['comedy'
'drama'
'music']
['drama'
'romance'
'war']
['crime'
'drama'
'mystery'
'thriller']
['adventure'
'drama'
'fantasy']
['comedy'
'crime'
'romance'
'thriller']
['drama']
['actio

'romance']
['drama'
'mystery']
['comedy']
['action'
'adventure'
'thriller']
['crime'
'drama'
'sci-fi']
['crime'
'drama'
'thriller']
['action'
'crime'
'mystery'
'romance'
'thriller']
['drama'
'mystery'
'sci-fi'
'thriller']
['crime'
'drama']
['crime'
'mystery'
'thriller']
['crime'
'drama']
['documentary']
['action'
'comedy'
'crime']
['action']
['action'
'crime'
'fantasy'
'thriller']
['drama'
'romance'
'thriller']
['drama'
'fantasy'
'horror']
['sci-fi'
'horror']
['mystery'
'sci-fi'
'thriller']
['comedy'
'romance']
['horror'
'sci-fi']
['drama']
['action'
'horror'
'sci-fi'
'thriller']
['drama'
'war']
['action'
'crime'
'sci-fi']
['comedy'
'drama'
'fantasy']
['action'
'crime'
'thriller']
['crime'
'drama']
['action'
'crime'
'drama'
'thriller']
['drama']
['comedy']
['action'
'adventure'
'sci-fi']
['biography'
'comedy'
'drama']
['comedy']
['comedy'
'drama']
['crime'
'drama'
'mystery'
'thriller']
['action'
'drama'
'thriller']
['romance'
'drama'
'war']
['action'
'crime'
'romance'
'thriller']
['bio

['action'
'crime'
'fantasy'
'mystery'
'sci-fi'
'thriller']
['action'
'adventure'
'crime']
['action'
'sci-fi']
['comedy'
'sci-fi']
['action'
'adventure'
'drama'
'history'
'war']
[]


In [11]:
genre

["['comedy' 'romance']",
 "['adventure' 'biography' 'drama' 'history']",
 "['action' 'crime' 'drama' 'thriller']",
 "['adventure' 'mystery' 'sci-fi']",
 "['action' 'comedy' 'crime' 'drama' 'thriller']",
 "['action' 'adventure' 'romance' 'sci-fi' 'thriller']",
 "['crime' 'mystery' 'thriller']",
 "['fantasy' 'horror' 'thriller']",
 "['fantasy' 'horror' 'thriller']",
 "['sci-fi' 'thriller']",
 "['drama' 'mystery' 'thriller']",
 "['action' 'drama' 'thriller']",
 "['comedy' 'romance' 'sci-fi']",
 "['comedy' 'romance']",
 "['crime' 'drama' 'sci-fi' 'thriller']",
 "['action' 'sci-fi' 'thriller']",
 "['biography' 'drama' 'music']",
 "['horror' 'romance']",
 "['drama']",
 "['action' 'western']",
 "['drama' 'thriller']",
 "['drama' 'thriller']",
 "['action' 'adventure' 'comedy' 'crime']",
 "['action' 'adventure' 'thriller']",
 "['comedy' 'romance']",
 "['action' 'crime' 'drama' 'mystery' 'thriller']",
 "['crime' 'drama']",
 "['comedy' 'drama' 'music']",
 "['drama' 'romance' 'war']",
 "['crime' '

In [12]:
genre_split = [g.split(' ') for g in genre]

In [13]:
genre_split

[["['comedy'", "'romance']"],
 ["['adventure'", "'biography'", "'drama'", "'history']"],
 ["['action'", "'crime'", "'drama'", "'thriller']"],
 ["['adventure'", "'mystery'", "'sci-fi']"],
 ["['action'", "'comedy'", "'crime'", "'drama'", "'thriller']"],
 ["['action'", "'adventure'", "'romance'", "'sci-fi'", "'thriller']"],
 ["['crime'", "'mystery'", "'thriller']"],
 ["['fantasy'", "'horror'", "'thriller']"],
 ["['fantasy'", "'horror'", "'thriller']"],
 ["['sci-fi'", "'thriller']"],
 ["['drama'", "'mystery'", "'thriller']"],
 ["['action'", "'drama'", "'thriller']"],
 ["['comedy'", "'romance'", "'sci-fi']"],
 ["['comedy'", "'romance']"],
 ["['crime'", "'drama'", "'sci-fi'", "'thriller']"],
 ["['action'", "'sci-fi'", "'thriller']"],
 ["['biography'", "'drama'", "'music']"],
 ["['horror'", "'romance']"],
 ["['drama']"],
 ["['action'", "'western']"],
 ["['drama'", "'thriller']"],
 ["['drama'", "'thriller']"],
 ["['action'", "'adventure'", "'comedy'", "'crime']"],
 ["['action'", "'adventure'",

In [19]:
[g for g in genre_split]

[["['comedy'", "'romance']"],
 ["['adventure'", "'biography'", "'drama'", "'history']"],
 ["['action'", "'crime'", "'drama'", "'thriller']"],
 ["['adventure'", "'mystery'", "'sci-fi']"],
 ["['action'", "'comedy'", "'crime'", "'drama'", "'thriller']"],
 ["['action'", "'adventure'", "'romance'", "'sci-fi'", "'thriller']"],
 ["['crime'", "'mystery'", "'thriller']"],
 ["['fantasy'", "'horror'", "'thriller']"],
 ["['fantasy'", "'horror'", "'thriller']"],
 ["['sci-fi'", "'thriller']"],
 ["['drama'", "'mystery'", "'thriller']"],
 ["['action'", "'drama'", "'thriller']"],
 ["['comedy'", "'romance'", "'sci-fi']"],
 ["['comedy'", "'romance']"],
 ["['crime'", "'drama'", "'sci-fi'", "'thriller']"],
 ["['action'", "'sci-fi'", "'thriller']"],
 ["['biography'", "'drama'", "'music']"],
 ["['horror'", "'romance']"],
 ["['drama']"],
 ["['action'", "'western']"],
 ["['drama'", "'thriller']"],
 ["['drama'", "'thriller']"],
 ["['action'", "'adventure'", "'comedy'", "'crime']"],
 ["['action'", "'adventure'",